# Recherche meilleurs algos

## import librairies et Pré-traitements 

In [1]:
# Importer les bibliothèques nécessaires

import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, confusion_matrix,accuracy_score
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import missingno as msno
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns
import imblearn as imb
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from imblearn import FunctionSampler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base=pd.read_csv("/content/drive/MyDrive/Notebooks/P7/basep7.csv")
        
except ModuleNotFoundError :
    
    base = pd.read_csv("basep7.csv")
    
base = base.drop( columns = ['Unnamed: 0'])

In [3]:
# one_hot_encoder classique pour les non numériques
def one_hot_encoder(base, nan_as_category = True):
    original_columns = list(base.columns)
    categorical_columns = [col for col in base.columns if base[col].dtype == 'object']
    base2 = pd.get_dummies(base, columns= categorical_columns, dummy_na= True)
    new_columns = [c for c in base.columns if c not in original_columns]
    return base2
print (base.shape)
base2 =one_hot_encoder(base)
print (base2.shape)
base = base2

(307511, 122)
(307511, 262)


In [4]:
# Remplacer les valeurs manquantes par la moyenne de la colonne
base = base.fillna(base.mean())

In [5]:
# Séparer les variables explicatives (X) et la variable cible (y)
X = base.drop("TARGET", axis=1)
y = base["TARGET"]



In [6]:
# scaler 
col_names=X.select_dtypes(include='number').columns.tolist()
features = X[col_names]
scaler = StandardScaler().fit(features.values)
features_scale = scaler.transform(features.values)
X[col_names] = features_scale
print (X.shape) 
del features
del features_scale

(307511, 261)


In [7]:
# Améliore l'affichage des DataFrame de résultats. 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# fonction comparer algos 


In [8]:
def comp_algos(X,y,algos,params,cvi,scorer,sampling) : 

    
    roc_aucs = []
    confusions = []
    Report_df = pd.DataFrame(columns=['Algorithme', 'score nom ','score']) 
    Append = []

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
    del X
    del y
    
    # Apply oversampling
    #ros = RandomOverSampler(random_state=42)
    #X_resampled, y_resampled = sampling.fit_resample(X_train,y_train)
    
    for i in range(len(algos)): 
        
        #faire boucle sur le sampling 
        for s in range(len(sampling)): 
            
            # Apply sampling
            
            debut = time.time()
            X_resampled, y_resampled = sampling[s].fit_resample(X_train,y_train)

# Créer un objet GridSearchCV qui va comparer les performances des algorithmes sur les données en utilisant le score roc_auc comme critère d'évaluation 
            print (algos[i], params[i] )
    
          
           
            grid = GridSearchCV(estimator=algos[i], param_grid=params[i], scoring=scorer, cv=cvi, refit= True )
          
            grid.fit(X_resampled,y_resampled)
            del X_resampled
            del y_resampled
            
# Afficher le meilleur score et le meilleur algorithme trouvé par GridSearchCV 
            print("Meilleur score:", grid.best_score_)
            print("Meilleur algorithme:", grid.best_estimator_)
            #print ( "cv results ;", grid.cv_results_ ) 
            print ( "best_params_",grid.best_params_)
            print ("best_index_",grid.best_index_)
            print ("scorer_",grid.scorer_)
            print ("n_splits_",grid.n_splits_)
            print ("sampling method",sampling[s])
    
            cv_result= pd.DataFrame(grid.cv_results_)
            display ( cv_result )

# Prédire les classes pour chaque algorithme 
            pred_y_train = grid.predict(X_train)
            score = accuracy_score(y_train,pred_y_train)
            print ('accuracy train',score)
          
            roc_auc_train = roc_auc_score(y_train, pred_y_train)
            #print ( 'roc_auc_train',roc_auc_train)
            print(f'roc_auc_train: {roc_auc_train:.1%}')

            pred_y_train_proba=grid.predict_proba(X_train)[:,1]
            roc_auc_discret_train = roc_auc_score(y_train, pred_y_train_proba)
            print( "Roc auc train predict proba",roc_auc_discret_train)
        
            pred_y_test = grid.predict(X_test)
            score = accuracy_score(y_test,pred_y_test)
            print ('accuracy test',score)
          
            roc_auc_test = roc_auc_score(y_test, pred_y_test)
            print ( 'roc_auc_ test ',roc_auc_test)

            pred_y_test_proba=grid.predict_proba(X_test)[:,1]
            roc_auc_discret_test= roc_auc_score(y_test, pred_y_test_proba)
            print( "Roc auc score test predict proba",roc_auc_discret_test)

# Calculer la matrice de confusion pour chaque algorithme

            matrice_confusion = confusion_matrix(y_true=y_train,y_pred=pred_y_train)
            tn, fp, fn, tp = confusion_matrix(y_true=y_train,y_pred=pred_y_train, normalize='all').ravel()

# Change figure size and increase dpi for better resolution
            plt.figure(figsize=(2,1), dpi=100)
# Scale up the size of all text
            sns.set(font_scale = 1)

# Plot Confusion Matrix using Seaborn heatmap()
# Parameters:
# first param - confusion matrix in array format   
# annot = True: show the numbers in each heatmap cell
# fmt = 'd': show numbers as integers. 
            ax = sns.heatmap(matrice_confusion, annot=True, fmt='d', )

# set x-axis label and ticks. 
            ax.set_xlabel("Prédiction ", fontsize=14, labelpad=20)
            ax.xaxis.set_ticklabels(['Accepté', 'Refus'])

# set y-axis label and ticks
            ax.set_ylabel("réel ", fontsize=14, labelpad=20)
            ax.yaxis.set_ticklabels(['Accepté', 'Refus'])

# set plot title
            titre = str(algos[i])
            ax.set_title(titre, fontsize=14, pad=20)
            plt.show()    
     
    
#graphique

            roc_auc_courbe = roc_auc_score(y_train, pred_y_train_proba)
            print ('roc auc courbe',roc_auc_courbe)
            fpr,tpr,_= roc_curve(y_train,pred_y_train_proba)
    
            plt.plot(fpr,tpr,label ='score (auc roc = %0.2f)' % roc_auc_courbe)
            plt.plot([0,1],[0,1],'k--',label ="random")
            plt.xlabel('Taux faux positifs')
            plt.ylabel('Taux vrais positifs')
            plt.title('ROC curve '+titre)
            plt.legend(loc = 'lower right')
            plt.show()
            
           
            
            # Créer un dataframe pandas qui contient les résultats de chaque algorithme
            print(' score ', scorer,':' ,grid.best_score_)
            duration = (time.time() - debut )/60
            print (duration ,'minutes')
            Append = pd.DataFrame([[ algos[i],grid.best_params_,scorer ,grid.best_score_,sampling[s],roc_auc_train,roc_auc_discret_train
                                    ,roc_auc_test,roc_auc_discret_test,tn,fp,fn,tp,duration]]
                                  ,columns=['Algorithme','Meilleur param', 'score nom ','best score','sampling','roc_auc_train'
                                            ,'Roc_auc_train_discret','roc_auc_test','roc_auc_discret_test','TN','FP','FN','TP','minutes'])
            Report_df = pd.concat([Report_df, Append]) 
            
    return Report_df

In [ ]:
algos = [ LogisticRegression(random_state=22),DecisionTreeClassifier(random_state=22), RandomForestClassifier(random_state=22), 
         XGBClassifier(random_state=22), GaussianNB(),DummyClassifier()]

# Définir une grille de paramètres à optimiser pour chaque algorithme
params = [
    
    {"C": [0.01, 0.1, 1, 10], "penalty": ["l1", "l2", "elasticnet", 'none']}, # pour LogisticRegression 
    {"max_depth": [3, 5, 7], "min_samples_leaf": [5, 10, 15]}, # pour DecisionTreeClassifier
    {"n_estimators": [100, 200], "max_depth": [3, 5]}, # pour RandomForestClassifier 
    {"n_estimators": [100, 200], "learning_rate": [0.01, 0.1]}, # pour XGBClassifier 
    {} ,# pas de paramètres à optimiser pour GaussianNB 
    {"strategy": ["most_frequent","prior","stratified","uniform"]}, # pour DummyClassifier
] 


cvi =2
scorer = "roc_auc" 
def nosamplerfunc(X, y):
    return X[:], y[:]
nosampler = FunctionSampler(func=nosamplerfunc)


sampling = [nosampler, RandomOverSampler(random_state=22),RandomUnderSampler(random_state=22),SMOTETomek(random_state=22)]



#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
result_auc= comp_algos(X,y,algos,params,cvi,scorer,sampling)

result_auc
result_auc.to_excel("resultat_auc_200323.xlsx")

In [10]:
result_auc.to_excel("resultat_auc_200323.xlsx")

In [11]:
result_auc.to_csv("resultat_auc_200323.csv")

# optimisation scorer métier 

In [ ]:
 algos = [ LogisticRegression(random_state=22),DecisionTreeClassifier(random_state=22), RandomForestClassifier(random_state=22), 
         XGBClassifier(random_state=22), GaussianNB(),DummyClassifier()]

# Définir une grille de paramètres à optimiser pour chaque algorithme
params = [
    
    {"C": [0.01, 0.1, 1, 10], "penalty": ["l1", "l2", "elasticnet", None]}, # pour LogisticRegression ‘l1’, ‘l2’, ‘elasticnet’, None
    {"max_depth": [3, 5, 7], "min_samples_leaf": [5, 10, 15]}, # pour DecisionTreeClassifier
    {"n_estimators": [100, 200], "max_depth": [3, 5]}, # pour RandomForestClassifier 
    {"n_estimators": [100, 200], "learning_rate": [0.01, 0.1]}, # pour XGBClassifier 
    {} ,# pas de paramètres à optimiser pour GaussianNB 
    {"strategy": ["most_frequent","prior","stratified","uniform"]}, # pour DummyClassifier
]

cvi =3

"""algos = [ DecisionTreeClassifier(random_state=22)]
params = [ {"max_depth": [3], "min_samples_leaf": [5]}] # pour DecisionTreeClassifier"""

def custom_score_func(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    score = tp + tn - fp -( 10 * fn)
    #score 2 = tp + (10 * tn )- fp -(10 * fn)
    return score


# Création du scorer personnalisé
score_metier= make_scorer(custom_score_func)

scorer = score_metier

def nosamplerfunc(X, y):
    return X[:], y[:]
nosampler = FunctionSampler(func=nosamplerfunc)

sampling = [nosampler, RandomOverSampler(random_state=22),RandomUnderSampler(random_state=22),SMOTETomek(random_state=22)]



result_metier= comp_algos(X,y,algos,params,cvi,scorer,sampling)
result_metier

result_metier.to_excel("resultat_metier_200323.xlsx")

In [13]:
#formatted_time = time.strftime("%Y-%m-%d-%H:%M:%S", time.localtime(time.time()))
#"nom_excel='result_metier_'+ formatted_time +'.xlsx'
result_metier.to_excel('score_metier_20323.xlsx')

In [14]:
result_metier.to_csv('score_metier_20323.csv')

Les courbes ROC permettent d’évaluer les performances d’un modèle de classification en fonction du seuil de décision choisi. Elles représentent le taux de vrais positifs (TPR) en fonction du taux de faux positifs (FPR) pour différents seuils. Plus la courbe est proche du coin supérieur gauche du graphique, plus le modèle est performant. L’aire sous la courbe (AUC) est une mesure synthétique qui résume la qualité d’un modèle: plus elle est proche de 1, mieux c’est.

